<a href="https://colab.research.google.com/github/brunoneri/KerasOptimizers/blob/master/keras_optimizer_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.0.0


In [0]:

import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

import numpy as np

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt



In [0]:
def plot_diagnostic(history, opt_label):
  train_label='{} train'.format(opt_label)
  val_label= '{} val'.format(opt_label)
  #plot loss  
  plt.figure(figsize=(10,5))
  plt.title('Cross Entropy Loss') 
  plt.xlabel('epoch')
  plt.ylabel('loss') 
  plt.plot(history.history['loss'], color='pink', label= train_label)
  plt.plot(history.history['val_loss'], color='gray', label=val_label)
  plt.legend([train_label,val_label], loc='upper right')
  # plot accuracy
  plt.show()
  plt.figure(figsize=(10,5))
  plt.title('Classification Accuracy')
  plt.xlabel('epochs')
  plt.ylabel('accuracy')
  plt.plot(history.history['accuracy'], color='pink', label=train_label)
  plt.plot(history.history['val_accuracy'], color='gray', label=val_label)
  plt.legend([train_label,val_label], loc='lower right')
  plt.show()

Caricamento del dataset Fashion MINST  

In [0]:
NUM_CLASSES = 10

(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
#Normalizzazione delle dati delle immagini 
x_train= x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0


Preparazione dei training e validation set.

In [0]:
#Splitting del training set: i primi 55000 esempi di training e gli 5000 esempi di validazione
(x_train, x_valid) = x_train[5000:], x_train[:5000] 
(y_train, y_valid) = y_train[5000:], y_train[:5000]

#Reshaping del dataset per adattarlo all'input layer delle CNN
w, h = 28, 28
x_train = x_train.reshape(x_train.shape[0], w, h, 1)
x_valid = x_valid.reshape(x_valid.shape[0], w, h, 1)
x_test = x_test.reshape(x_test.shape[0], w, h, 1)

#One-hot-encoding delle label
y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
y_valid = tf.keras.utils.to_categorical(y_valid, NUM_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)


print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)


print(x_train.shape[0], 'train set')
print(x_valid.shape[0], 'validation set')
print(x_test.shape[0], 'test set')

Implementazione del modello di Convolutional neural network

In [0]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    #Primo layer convoluzionale
    keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1)),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Dropout(0.3),
    #Secondo layer convoluzionale
    keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Dropout(0.3),
    #Flattening
    keras.layers.Flatten(),
    #Full connection layer
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

model.summary()

Addestramento della CNN con l'utilizzo dell'ottimizzatore SGD

In [0]:
model.compile(loss="categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=0.001),
              metrics=["accuracy"])


In [0]:
history_sgd = model.fit(x_train, y_train, epochs=10,batch_size=64,
                    validation_data=(x_valid, y_valid))

In [0]:
plot_diagnostic(history_sgd,"sgd")

Addestramento della CNN con l'utilizzo dell'ottimizzatore SGD e del momentum

In [0]:
model.compile(loss="categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.9),
              metrics=["accuracy"])

In [0]:
history_momentum = model.fit(x_train, y_train, epochs=10,batch_size=64,
                    validation_data=(x_valid, y_valid))

In [0]:
plot_diagnostic(history_momentum,"Momentum")

Addestramento della CNN con l'utilizzo dell'ottimizzatore SGD ed il momento di 
Nesterov


In [0]:
model.compile(loss="categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True),
              metrics=["accuracy"])


In [0]:
history_momentum_nesterov = model.fit(x_train, y_train, epochs=10,batch_size=64,
                    validation_data=(x_valid, y_valid))


In [0]:
plot_diagnostic(history_momentum_nesterov,"Nesterov")

Addestramento della CNN con l'utilizzo dell'ottimizzatore RMSProp 

In [0]:
model.compile(loss="categorical_crossentropy",
              optimizer=keras.optimizers.RMSprop(lr=0.001, rho=0.9),
              metrics=["accuracy"])


In [0]:
history_rmsprop = model.fit(x_train, y_train, epochs=10,batch_size=64,
                    validation_data=(x_valid, y_valid))


In [0]:
plot_diagnostic(history_rmsprop,"RMSProp")

Addestramento della CNN con l'utilizzo dell'ottimizzatore Adam

In [0]:
model.compile(loss="categorical_crossentropy",
              optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
              metrics=["accuracy"])


In [0]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose = 1, save_best_only=True)

history_adam = model.fit(x_train, y_train, epochs=10,batch_size=64,
                    validation_data=(x_valid, y_valid), callbacks=[checkpointer])


In [0]:
plot_diagnostic(history_adam,"Adam")

Visualizzazione delle curve di apprendimento (loss e accuracy)

In [0]:
# plot learning curves
plt.figure(figsize=(10,5))
plt.title('Cross Entropy Loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(history_sgd.history['loss'], color='blue', label='SGD train')
plt.plot(history_sgd.history['val_loss'], color='orange', label='SGD val')
plt.plot(history_momentum.history['loss'], color='green', label='Momentum train')
plt.plot(history_momentum.history['val_loss'], color='red', label='Momentum val')
plt.plot(history_momentum_nesterov.history['loss'], color='purple', label='Nesterov train')
plt.plot(history_momentum_nesterov.history['val_loss'], color='brown', label='Nesterov val')
plt.plot(history_rmsprop.history['loss'], color='pink', label='RMSProp train')
plt.plot(history_rmsprop.history['val_loss'], color='gray', label='RMSProp val')
plt.plot(history_adam.history['loss'], color='olive', label='Adam train')
plt.plot(history_adam.history['val_loss'], color='cyan', label='Adam val')
plt.legend(['SGD train', 'SGD val','Momentum train','Momentum val','Nesterov train', 'Nesterov val','RMSProp train','RMSProp val','Adam train', 'Adam val'], loc='upper right')
# plot accuracy
plt.show()
plt.figure(figsize=(10,5))
plt.title('Classification Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.plot(history_sgd.history['accuracy'], color='blue', label='SGD train')
plt.plot(history_sgd.history['val_accuracy'], color='orange', label='SGD val')
plt.plot(history_momentum.history['accuracy'], color='green', label='Momentum train')
plt.plot(history_momentum.history['val_accuracy'], color='red', label='Momentum val')
plt.plot(history_momentum_nesterov.history['accuracy'], color='purple', label='Nesterov train')
plt.plot(history_momentum_nesterov.history['val_accuracy'], color='brown', label='Nesterov val')
plt.plot(history_rmsprop.history['accuracy'], color='pink', label='RMSProp train')
plt.plot(history_rmsprop.history['val_accuracy'], color='gray', label='RMSProp val')
plt.plot(history_adam.history['accuracy'], color='olive', label='Adam train')
plt.plot(history_adam.history['val_accuracy'], color='cyan', label='Adam val')
plt.legend(['SGD train', 'SGD val','Momentum train','Momentum val','Nesterov train', 'Nesterov val','RMSProp train','RMSProp val','Adam train', 'Adam val'], loc='lower right')
plt.show()

Valutazione del modello mediante il test set. 

In [0]:

model.load_weights('model.weights.best.hdf5')

In [0]:
# Valutazione del modello con il test set
score = model.evaluate(x_test, y_test, verbose=0)

print('\n', 'Test accuracy:', score[1])

In [0]:
# Etichette del dataset
fashion_mnist_labels = ["T-shirt/top",  # index 0
                        "Trouser",      # index 1
                        "Pullover",     # index 2 
                        "Dress",        # index 3 
                        "Coat",         # index 4
                        "Sandal",       # index 5
                        "Shirt",        # index 6 
                        "Sneaker",      # index 7 
                        "Bag",          # index 8 
                        "Ankle boot"]   # index 9


y_hat = model.predict(x_test)

# Visualizzazine di 10 immagini campione del dataset di test con il relativo risultato della classificazione
figure = plt.figure(figsize=(20, 8))
for i, index in enumerate(np.random.choice(x_test.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_test[index]))
    predict_index = np.argmax(y_hat[index])
    true_index = np.argmax(y_test[index])
  
    ax.set_title("{} ({})".format(fashion_mnist_labels[predict_index], 
                                  fashion_mnist_labels[true_index]),
                                  color=("green" if predict_index == true_index else "red"))